<a href="https://colab.research.google.com/github/cbittel/Coding_Challenge/blob/main/Section1_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask pyngrok


from flask import Flask, request, jsonify
from pyngrok import ngrok
import threading
import io
import pandas as pd
import sqlite3


In [2]:
database_name = 'historical_data.db'


In [3]:

def clear_database():
  try:
    db_connection = sqlite3.connect(database_name)
    cursor = db_connection.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = cursor.fetchall()
    for table in tables:
        cursor.execute(f"DROP TABLE {table[0]}")
    db_connection.commit()
    db_connection.close()
  except Exception as e:
    return str(e)

In [4]:

# DEfine Flask application interface
app = Flask(__name__)

def run_flask_app():
    app.run()


In [5]:

@app.route('/hello', methods=['GET'])
def hello():
    return jsonify(message="Hello from your API!")


In [6]:
@app.route('/historical_data_csv', methods=['POST'])
def historical_data_csv():
    """
    Endpoint that receives a list of three CSV files and saves them to a SQLite3 database.
    """

    try:
        files = request.files.getlist('files')

        if len(files) != 3:
            return jsonify(error="Exactly three files are required")

        # Clear the SQLite database before processing
        clear_database()
        processed_data = []

        for file in files:
            if file.filename == '':
                return jsonify(error="One of the files is empty")
            elif file.filename == 'departments.csv':
              column_names = ['dept_id', 'dept_name']
            elif file.filename == 'jobs.csv':
              column_names = ['job_id', 'job_name']
            elif file.filename == 'hired_employees.csv':
              column_names = ['empl_id', 'empl_name', 'hired', 'dept_id', 'job_id']

            # Read the CSV file into a pandas DataFrame
            csv_data = file.read().decode('utf-8')
            df = pd.read_csv(io.StringIO(csv_data), names=column_names)   # pd.compat.StringIO(csv_data)

            # Connect to SQLite database
            db_connection = sqlite3.connect(database_name)
            # Get table name from the filename without extension
            table_name = file.filename.split('.')[0]

            batch_size = 1000
            num_batches = len(df) // batch_size + 1

            for batch_num in range(num_batches):
                batch_df = df.iloc[batch_num * batch_size : (batch_num + 1) * batch_size]
                # Process the batch - example: convert to JSON format
                processed_batch = batch_df.to_json(orient='records')
                # Process the batch - save to SQLite3 format
                batch_df.to_sql(table_name, db_connection, if_exists='append', index=False)

                processed_data.append(processed_batch)

            db_connection.close()
            print(jsonify(processed_data.append("Data successfully processed and saved")))

        return jsonify(processed_data)
    except Exception as e:
        return jsonify(error=str(e))


In [7]:

# Start the Flask app in a separate thread
flask_thread = threading.Thread(target=run_flask_app)
flask_thread.start()


 * Serving Flask app '__main__'


In [8]:

# Get the public URL using ngrok
public_url = ngrok.connect(addr='5000')
print(f'public_url: {public_url}     ---   copy this URL to the tests module "public_url" variable.')


 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


public_url: NgrokTunnel: "https://25b6-35-236-254-50.ngrok.io" -> "http://localhost:5000"     ---   copy this URL to the tests module "public_url" variable.


In [12]:
!ls

drive  historical_data.db  sample_data


In [13]:
import sqlite3
import os

# Specify the path on your Google Drive where the database is located
drive_path = '/content/drive/MyDrive/Data Science/'
db_path = os.path.join(drive_path, database_name)

# Connect to the database
db_connection = sqlite3.connect(database_name)

# Execute a SELECT query
cursor = db_connection.cursor()
select_query = 'SELECT * FROM hired_employees'
cursor.execute(select_query)
results = cursor.fetchall()
for row in results:
    print(row)

# Close the connection
db_connection.close()


In [14]:
results

[]